In [80]:
import sys
print(sys.executable)

/Users/bipinjethwani/my_codebases/langgraph_playbook/.venv/bin/python3


In [81]:
from openai import OpenAI
import re

from dotenv import load_dotenv
_ = load_dotenv()

client = OpenAI()

In [82]:
# Define a prompt for the agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE, and don't process anything further until you get the result back.
Observation will be the result of running those actions, which will be returned in the next message.

Your available actions are:

calculate_total_price:
e.g. calculate_total_price: apple: 2, banana: 3
Runs a calculation for the total price based on the quantity and prices of the fruits.

get_fruit_price:
e.g. get_fruit_price: apple
returns the price of the fruit when given its name.

Example session:

Question: What is the total price for 2 apples and 3 bananas?
Thought: I should calculate the total price by getting the price of each fruit and summing them up.
Action: get_fruit_price: apple
PAUSE

Observation: The price of an apple is $1.5.

Action: get_fruit_price: banana
PAUSE

Observation: The price of a banana is $1.2.

Action: calculate_total_price: apple: 2, banana: 3
PAUSE

You then output:

Answer: The total price for 2 apples and 3 bananas is $6.6.
""".strip()
prompt

"You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE, and don't process anything further until you get the result back.\nObservation will be the result of running those actions, which will be returned in the next message.\n\nYour available actions are:\n\ncalculate_total_price:\ne.g. calculate_total_price: apple: 2, banana: 3\nRuns a calculation for the total price based on the quantity and prices of the fruits.\n\nget_fruit_price:\ne.g. get_fruit_price: apple\nreturns the price of the fruit when given its name.\n\nExample session:\n\nQuestion: What is the total price for 2 apples and 3 bananas?\nThought: I should calculate the total price by getting the price of each fruit and summing them up.\nAction: get_fruit_price: apple\nPAUSE\n\nObservation: The price of an apple is $1.5.\n\nA

In [83]:
class Agent:
    """AI agent that maintains conversation history with OpenAI's API."""

    def __init__(self, system: str = ""):
        self.system = system
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message: str) -> str:
        """Send a message and get a response."""
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})

        return result

    def execute(self) -> str:
        """Call the OpenAI API with the full conversation history."""
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

In [84]:
FRUIT_PRICES = {  # UPPERCASE for constants
    "apple": 1.5,
    "banana": 1.2,
    "orange": 1.3,
    "grapes": 2.0,
}

def get_fruit_price(fruit: str) -> str:
    """Look up the price of a single fruit."""
    fruit = fruit.lower().strip()  # Handle "Apple" or " apple "
    if fruit in FRUIT_PRICES:
        return f"The price of {fruit} is ${FRUIT_PRICES[fruit]:.2f}"
    return f"Unknown fruit: {fruit}. Available: {', '.join(FRUIT_PRICES.keys())}"

def calculate_total_price(fruits: str) -> str:
    """Calculate total for 'apple: 2, banana: 3' format."""
    total = 0.0
    for item in fruits.split(","):
        try:
            fruit, quantity = item.split(":")
            fruit = fruit.strip().lower()
            quantity = int(quantity.strip())
            if fruit not in FRUIT_PRICES:
                return f"Unknown fruit: {fruit}"
            total += FRUIT_PRICES[fruit] * quantity
        except ValueError:
            return f"Invalid format: '{item}'. Use 'fruit: quantity'"
    return f"The total price is ${total:.2f}"

known_actions = {
    "get_fruit_price": get_fruit_price,
    "calculate_total_price": calculate_total_price,
}

In [85]:
react_agent = Agent(system=prompt)

In [86]:
# Run a query
action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to select action

def query(question):
    bot = Agent(prompt)
    result = bot(question)
    print("LLM: \n--\n", result, "\n--")
    actions = [
        action_re.match(a) 
        for a in result.split('\n') 
        if action_re.match(a)
    ]
    if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
            raise Exception(f"Unknown action: {action}: {action_input}")
        print(f" --> running {action} {action_input}")
        observation = known_actions[action](action_input)
        print("Observation:", observation)
    else:
        return


In [87]:
# APPROACH 2 (BEST PRACTICE): Strict one-action-at-a-time loop

def query_strict(question, max_turns=10):
    """Execute ONE action per turn, feed observation, let LLM continue. True ReAct loop."""
    bot = Agent(prompt)
    next_prompt = question
    
    for turn in range(max_turns):
        result = bot(next_prompt)
        print(f"=== Turn {turn + 1} ===")
        print("LLM:\n", result, "\n")
        
        # Check if we have a final answer
        if "Answer:" in result and "PAUSE" not in result:
            print("✅ Final answer reached!")
            return
        
        # Find the FIRST action only
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        
        if not actions:
            print("No action found, stopping.")
            return
        
        # Execute only the FIRST action
        action, action_input = actions[0].groups()
        if action not in known_actions:
            raise Exception(f"Unknown action: {action}: {action_input}")
        
        print(f" --> running {action}({action_input})")
        observation = known_actions[action](action_input)
        print(f"     Observation: {observation}\n")
        
        # Feed ONE observation back — LLM will reason about next step
        next_prompt = f"Observation: {observation}"
    
    print("⚠️ Max turns reached without final answer")

In [88]:
query("What is the price of a banana?")

LLM: 
--
 Thought: I need to find out the price of a banana by using the appropriate action to get its price. 
Action: get_fruit_price: banana
PAUSE 
--
 --> running get_fruit_price banana
Observation: The price of banana is $1.20


In [89]:
query("What is the price of a papaya?")

LLM: 
--
 Thought: I need to find out the price of a papaya by using the get_fruit_price action. 
Action: get_fruit_price: papaya
PAUSE 
--
 --> running get_fruit_price papaya
Observation: Unknown fruit: papaya. Available: apple, banana, orange, grapes


In [90]:
query("What is the price of a orange?")

LLM: 
--
 Thought: I need to find out the price of an orange by using the get_fruit_price action. 
Action: get_fruit_price: orange
PAUSE 
--
 --> running get_fruit_price orange
Observation: The price of orange is $1.30


In [91]:
query("What is the price of a banana and a orange?")

LLM: 
--
 Thought: I need to find the price of both a banana and an orange. I will start by getting the price of a banana first. 
Action: get_fruit_price: banana
PAUSE 
--
 --> running get_fruit_price banana
Observation: The price of banana is $1.20


In [92]:
query("What is the price of a banana, a orange, and an apple?")

LLM: 
--
 Thought: I need to find the price of each fruit: a banana, an orange, and an apple. I will start by getting the price of a banana. 
Action: get_fruit_price: banana
PAUSE 
--
 --> running get_fruit_price banana
Observation: The price of banana is $1.20


In [93]:
print("=" * 60)
print("APPROACH 2: Strict ReAct (one action at a time) - BEST PRACTICE")
print("=" * 60)
query_strict("What is the price of a banana, an orange, and an apple?")

APPROACH 2: Strict ReAct (one action at a time) - BEST PRACTICE
=== Turn 1 ===
LLM:
 Thought: I need to find the price of each fruit: a banana, an orange, and an apple. I will start by getting the price of a banana. 
Action: get_fruit_price: banana
PAUSE 

 --> running get_fruit_price(banana)
     Observation: The price of banana is $1.20

=== Turn 2 ===
LLM:
 Thought: I have the price of a banana. Now, I need to get the price of an orange. 
Action: get_fruit_price: orange
PAUSE 

 --> running get_fruit_price(orange)
     Observation: The price of orange is $1.30

=== Turn 3 ===
LLM:
 Thought: I now have the price of both a banana and an orange. Next, I need to get the price of an apple. 
Action: get_fruit_price: apple
PAUSE 

 --> running get_fruit_price(apple)
     Observation: The price of apple is $1.50

=== Turn 4 ===
LLM:
 Thought: I have now gathered the prices for all three fruits: a banana for $1.20, an orange for $1.30, and an apple for $1.50. I can now summarize the prices. 